In [1]:
%load_ext autoreload
%autoreload 2
from myst_nb import glue

# The basics

Before anything else, let's import the classes we need:

In [2]:
try:
    from respace import ResultSet
except ImportError:
    !pip install respace
    from respace import ResultSet

## The ResultSet class

Now let's build the simplest possible {class}`~respace.ResultSet`: it contains only one result `result` that depends on a single parameter `parameter`. And let's make it verbose, so we see more of what's going on:

In [3]:
def add_one(parameter): return parameter + 1
rs = ResultSet({"result": add_one}, {"parameter": 1}, verbose=True)
rs

<xarray.Dataset>
Dimensions:    (parameter: 1)
Coordinates:
  * parameter  (parameter) int64 1
Data variables:
    result     (parameter) int64 -1

Here you can see what's displayed is an {class}`xarray.Dataset` instance, which is a
representation of your parameter space (the {attr}`~respace.ResultSet.param_space` attribute of `rs`). You can see
all the data you've entered in there, except `add_one`, the computing function[^actually-in-res-attr]. So where
is it? Let's select the result and see what we got:

[^actually-in-res-attr]: Users familiar with `xarray` may know it can already be seen by expanding "result"'s attributes, but here we don't assume any prior knowledge of `xarray`, and anyway that's a nice way to introduce the `__getitem__` behaviour.

In [4]:
rs["result"]

<xarray.DataArray 'result' (parameter: 1)>
array([-1])
Coordinates:
  * parameter  (parameter) int64 1
Attributes:
    tracking_compute_fun:  <function add_one at 0x7f49fdfa3370>
    computed_values:       []
    compute_times:         []
    name:                  result
    compute_fun:           <function add_one at 0x7f4a39cb13f0>
    save_fun:              <function save_pickle at 0x7f49fdfa1510>
    load_fun:              <function load_pickle at 0x7f49fdfa1630>
    save_suffix:           .pickle
    save_path_fmt:         None

There it is, under `Attributes`! And there's some other stuff there too. But we'll get
back to that. First let's look at the "values" of the result displayed there: an array
with just a `-1`. But what is this doing there? Nothing was computed. Well, to find out,
let's {meth}`~respace.ResultSet.compute` a value of `result`:

In [5]:
res = rs.compute("result", {})
print(res)
rs["result"]

Computing result for the following parameter values:
{'parameter': 1}
2


<xarray.DataArray 'result' (parameter: 1)>
array([0])
Coordinates:
  * parameter  (parameter) int64 1
Attributes:
    tracking_compute_fun:  <function add_one at 0x7f49fdfa3370>
    computed_values:       [2]
    compute_times:         [3.5762786865234375e-06]
    name:                  result
    compute_fun:           <function add_one at 0x7f4a39cb13f0>
    save_fun:              <function save_pickle at 0x7f49fdfa1510>
    load_fun:              <function load_pickle at 0x7f49fdfa1630>
    save_suffix:           .pickle
    save_path_fmt:         None

Many things have changed in here, but let's start with the value in the array: it became
`0`. So what does it mean? It means that the result for `parameter = 1` is located at
index `0` of the `computed_values` attribute: there is the `2` resulting from the
addition. But how did it know to add `1 + 1`? Well since `parameter` was not provided in
the dictionary passed as second argument to `rs.compute()`, its default value was taken.
Here since the parameter has only one possible value, that's the default. Otherwise, the
first value along the `parameter` axis will be the default. Let's now add more parameter
values to see how that goes.

## Changing the parameters

In [6]:
rs.add_param_values({'parameter': [2, 3, 4]})
rs["result"]

<xarray.DataArray 'result' (parameter: 4)>
array([ 0, -1, -1, -1])
Coordinates:
  * parameter  (parameter) int64 1 2 3 4
Attributes:
    tracking_compute_fun:  <function add_one at 0x7f49fdfa3370>
    computed_values:       [2]
    compute_times:         [3.5762786865234375e-06]
    name:                  result
    compute_fun:           <function add_one at 0x7f4a39cb13f0>
    save_fun:              <function save_pickle at 0x7f49fdfa1510>
    load_fun:              <function load_pickle at 0x7f49fdfa1630>
    save_suffix:           .pickle
    save_path_fmt:         None

As you can see, the new parameter values are there in the `Coordinates`, and the array's size increased along the exising axis, with the `0` still at the coordinate corresponding to `parameter = 1`, and `-1` elsewhere. Let's see what happens if we now try to {meth}`~respace.ResultSet.get` the result as we did for `compute` above:

In [7]:
res = rs.get("result", {})
print(res)
rs["result"]

2


<xarray.DataArray 'result' (parameter: 4)>
array([ 0, -1, -1, -1])
Coordinates:
  * parameter  (parameter) int64 1 2 3 4
Attributes:
    tracking_compute_fun:  <function add_one at 0x7f49fdfa3370>
    computed_values:       [2]
    compute_times:         [3.5762786865234375e-06]
    name:                  result
    compute_fun:           <function add_one at 0x7f4a39cb13f0>
    save_fun:              <function save_pickle at 0x7f49fdfa1510>
    load_fun:              <function load_pickle at 0x7f49fdfa1630>
    save_suffix:           .pickle
    save_path_fmt:         None

Here's the `2` again, and we didn't get any message saying a new value was
computed. That's because it wasn't, since the value was already computed it was just
retrieved from the right position in `computed_values`. Now if we `get` for a different
`parameter` value:

In [8]:
res = rs.get("result", {"parameter": 3})
print(res)
rs["result"]

Computing result for the following parameter values:
{'parameter': 3}
4


<xarray.DataArray 'result' (parameter: 4)>
array([ 0, -1,  1, -1])
Coordinates:
  * parameter  (parameter) int64 1 2 3 4
Attributes:
    tracking_compute_fun:  <function add_one at 0x7f49fdfa3370>
    computed_values:       [2, 4]
    compute_times:         [3.5762786865234375e-06, 2.384185791015625e-06]
    name:                  result
    compute_fun:           <function add_one at 0x7f4a39cb13f0>
    save_fun:              <function save_pickle at 0x7f49fdfa1510>
    load_fun:              <function load_pickle at 0x7f49fdfa1630>
    save_suffix:           .pickle
    save_path_fmt:         None

In [9]:
glue("populated_space", rs.populated_space["result"])

<xarray.DataArray 'result' (parameter: 2)>
array([0, 1])
Coordinates:
  * parameter  (parameter) int64 1 3
Attributes:
    tracking_compute_fun:  <function add_one at 0x7f49fdfa3370>
    computed_values:       [2, 4]
    compute_times:         [3.5762786865234375e-06, 2.384185791015625e-06]
    name:                  result
    compute_fun:           <function add_one at 0x7f4a39cb13f0>
    save_fun:              <function save_pickle at 0x7f49fdfa1510>
    load_fun:              <function load_pickle at 0x7f49fdfa1630>
    save_suffix:           .pickle
    save_path_fmt:         None

As expected, here it computed the result for the new value.

`````{note}
Now if we want to
see only the part of the parameter space where values have been computed, we can use the
{attr}`~respace.ResultSet.populated_space` property:

```{code} python
rs.populated_space["result"]
```

````{toggle}

```{glue} populated_space
```

````

`````

And what happens if we try to make a computation for a parameter value that's not in the parameter space?

In [10]:
res = rs.get("result", {"parameter": 5})
print(res)
rs["result"]

Computing result for the following parameter values:
{'parameter': 5}
6


<xarray.DataArray 'result' (parameter: 5)>
array([ 0, -1,  1, -1,  2])
Coordinates:
  * parameter  (parameter) int64 1 2 3 4 5
Attributes:
    tracking_compute_fun:  <function add_one at 0x7f49fdfa3370>
    computed_values:       [2, 4, 6]
    compute_times:         [3.5762786865234375e-06, 2.384185791015625e-06, 2....
    name:                  result
    compute_fun:           <function add_one at 0x7f4a39cb13f0>
    save_fun:              <function save_pickle at 0x7f49fdfa1510>
    load_fun:              <function load_pickle at 0x7f49fdfa1630>
    save_suffix:           .pickle
    save_path_fmt:         None

Well it's simply added to the set and the computation goes through.

## Adding parameters

What if we need to add some new parameters at some point? That's what the {meth}`~respace.ResultSet.add_params` method is for. Here are different ways to use it that show some of the types of parameters that can be added:

In [11]:
from datetime import date
from respace import Parameter
rs.add_params({"date": [date(2000, 1, 1), date.today()], "constant": 4})
rs.add_params(Parameter("letter", default="c", values=["a", "b", "c"]))
rs["result"]

<xarray.DataArray 'result' (constant: 1, date: 2, letter: 3, parameter: 5)>
array([[[[ 0, -1,  1, -1,  2],
         [ 0, -1,  1, -1,  2],
         [ 0, -1,  1, -1,  2]],

        [[ 0, -1,  1, -1,  2],
         [ 0, -1,  1, -1,  2],
         [ 0, -1,  1, -1,  2]]]])
Coordinates:
  * letter     (letter) object 'c' 'a' 'b'
  * constant   (constant) int64 4
  * date       (date) object 2000-01-01 2023-02-20
  * parameter  (parameter) int64 1 2 3 4 5
Attributes:
    tracking_compute_fun:  <function add_one at 0x7f49fdfa3370>
    computed_values:       [2, 4, 6]
    compute_times:         [3.5762786865234375e-06, 2.384185791015625e-06, 2....
    name:                  result
    compute_fun:           <function add_one at 0x7f4a39cb13f0>
    save_fun:              <function save_pickle at 0x7f49fdfa1510>
    load_fun:              <function load_pickle at 0x7f49fdfa1630>
    save_suffix:           .pickle
    save_path_fmt:         None

```{note}
Note how dimensions have been added to the array, and how the default value for `"letter"` was shifted to the first position: that is so we always know which value is the default.
```

```{warning}
Beware that the existing result values are then assumed to have been computed for the default value of the added parameters. So you should always make sure (and that's usually a good programming practice!) that for the new parameters set at their default value, the behaviour of the computing function is unchanged. Also, if needed, don't forget to update it accordingly. If parameters are absent from the signature of the function, the default behaviour implemented in ReSpace is to silently ignore these parameters[^why-ignore].
```

[^why-ignore]: Why silently ignore irrelevant parameters? That's because a ResultSet is meant to hold multiple results depending potentially on different parameters. It would then be extremely annoying to have to update the computing function of every result every time a new parameter needs to be added to one result.

## Adding results

Equivalently, you have the {meth}`~respace.ResultSet.add_results` method to introduce
new results in the set. Here's how you use it:

In [12]:
from respace import ResultMetadata

rs.add_results({"other_result": lambda parameter, constant: parameter - constant})
rs.add_results([ResultMetadata("c", lambda: 1, save_path_fmt="c")])
rs

<xarray.Dataset>
Dimensions:       (letter: 3, constant: 1, date: 2, parameter: 5)
Coordinates:
  * letter        (letter) object 'c' 'a' 'b'
  * constant      (constant) int64 4
  * date          (date) object 2000-01-01 2023-02-20
  * parameter     (parameter) int64 1 2 3 4 5
Data variables:
    result        (constant, date, letter, parameter) int64 0 -1 1 -1 ... 1 -1 2
    other_result  (letter, constant, date, parameter) int64 -1 -1 -1 ... -1 -1
    c             (letter, constant, date, parameter) int64 -1 -1 -1 ... -1 -1

## Saving results

ReSpace also makes it super easy for you to save your results, let's have a look:

In [13]:
_ = rs.save("result", {"parameter": 5})
_ = rs.save("c", {})

Saving result at result_letter=c_constant=4_date=2000-01-01_parameter=5.pickle.
Computing c for the following parameter values:
{'letter': 'c', 'constant': 4, 'date': datetime.date(2000, 1, 1), 'parameter': 1}
Saving c at c.pickle.


The result `"c"` was saved according to the save path format we passed it. More
interestingly, `"result"` was saved at a path indicating first its name, and then a
string giving the name of the parameters and their values.